In [ ]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize
from scipy.stats import skew, kurtosis, kurtosistest, norm, t

import matplotlib.pyplot as plt
from matplotlib import cm
%matplotlib inline
import bokeh
from bokeh.io import output_notebook
from bokeh.layouts import column
from bokeh.palettes import inferno
from bokeh.plotting import figure, show
from bokeh.models import HoverTool, Legend, Span, Label
output_notebook()

import tqdm

Olá investidores,

Como acompanhamos, no último mês o mercado de criptomoedas vem sofrendo uma das maiores correções dos últimos tempos, resultando em uma queda de aproximadamente 57,8% do valor total de mercado (reduzindo de 829,5bi em 8 de janeiro a 349,6bi em apenas 30 dias, de acordo com https://coinmarketcap.com).

In [ ]:
data = pd.read_csv("../input/crypto-markets.csv")
marketcap_df = pd.pivot_table(data, index='date', columns='symbol', values='market',
                              fill_value=0.0).loc['2017-01-01':]

plt.figure(figsize=(13,6))
marketcap_df.sum(axis=1).divide(1000000000).plot()
plt.title("Capitalização do mercado")
plt.ylabel("valor ($ Bi)")
plt.xlabel("data")
plt.grid(linestyle='--', alpha=0.5);

Essa forte correção é mais provavelmente explicada pela sequência de notícias negativas com relação a esse mercado. A China temporariamente baniu as trocas de bens virtuais em setembro; no começo do ano a Coréia do Sul aprovou leis mais duras que dificultam o processo de compra e venda de criptomoedas nas casas de câmbio; o ministro de finanças da India deu uma entrevista onde se opôs fortemente ao uso de criptomoedas como forma de pagamento, além dos recentes casos de hackeamento reportados pela casa de câmbio japonesa Coincheck. Tudo isso somado à forte característica especulativa do mercado resultou na tendência de baixa apresentada recentemente.

Gerência de risco é um dos pontos mais importantes no desenvolvimento de estratégias de troca, especialmente quando o mercado apresenta fortes tendências de baixa. A idéia desse artigo é apresentar de uma forma simples técnicas eficientes de reduzir o risco ao montar um portifolio de investimento. Ao combinar diferentes técnicas de aversão ao risco com a estratégia de investimento obtemos um portifolio mais robusto às oscilações do mercado. É importante ressaltar que em mercados de alto risco e fortemente correlacionados, como é o caso das criptomoedas, o risco é inerente  e sem a devida diversificação para mercados descorrelacionados é impossível evitar por completo correções de valor.

Em momentos turbulentos como esse, cabe à nossa estratégia de troca assumir o menor risco possível dadas as limitações do mercado. Algumas técnicas de redução do risco serão apresentadas aqui, de forma a deixar claro ao investidor como é feito o posicionamento dos nossos fundos quando o mercado apresenta características de baixa. 
   
### Seleção de Universo   

Uma das decisões mais importantes em qualquer estratégia de troca é escolher quais ativos devem ser incluídos como opções de investimento. No momento, existem 70 moedas disponíveis para troca no nosso algoritmo. No entanto, uma seleção adequada dessas moedas não só aumenta a performance da estratégia como diminui significativamente o risco. Selecionamos semanalmente com cuidado as 20 melhores moedas de acordo com sua preponderância (valor de mercado, liquidez, volume de transações, diferença entre o preço de compra e de venda), a fim de manter apenas as moedas mais relevantes como opções de investimento e evitar portifolios com moedas cujos desempenhos vêm decaindo. De fato, atualmente as 20 maiores moedas representam 87,46% do valor de mercado de todas as moedas como mostrado abaixo.
 

In [ ]:
marketcap_df = marketcap_df.apply(lambda x: [x[i] / x.sum() for i in range(x.shape[0])],
                                  axis=1).astype('f')

smoothed_mc = marketcap_df.ewm(alpha=0.1).mean()

n = 20
largest = pd.DataFrame(index=marketcap_df.columns)
for i, row in enumerate(smoothed_mc.iterrows()):
    # choose pairs every quarter
    if i % int(7) == 0:
        components_index = np.argpartition(row[1].values, -n)[-n:]
    largest[row[0]] = row[1].iloc[components_index].fillna(0)
largest = largest.T
pie = largest.iloc[-1].dropna().sort_values()
pie['Outras moedas'] = 1 - largest.iloc[-1].sum()
fig1, ax1 = plt.subplots(figsize=(8,8))
ax1.pie(pie.values, labels = pie.index, radius=1.5, autopct='%.1f %%', pctdistance=0.8, shadow=True, explode=np.append(np.zeros(len(pie)-1),0.2))
ax1.axis('equal');

  
### Diversificação do Portifolio

Diversificação é uma forma efetiva de reduzir o risco. Um portifolio com uma variedade de investimentos é consideravelmente menos suscetível a grandes quedas, visto que a chance de todos os investimentos apresentarem uma performance ruim ao mesmo tempo é baixa. Na prática, os lucros e perdas de diferentes investimentos tendem a se opor e reduzir as variações no valor total do portifolio. Vale lembrar que o mercado de criptomoedas ainda apresenta grande correlação entre seus ativos, o que diminui o efeito da diversificação na redução do risco. Entretanto, o fato de as diferentes moedas apresentarem características e aplicações diversas gera uma tendência de independência entre elas, que consequentemente diminui o nivel de correlação com o tempo.

### Rebalanceamento

Rebalanceamento do portifolio é uma estratégia que ajuda o investidor a manter uma exposição constante ao universo de ativos a fim de extrair máxima performance com relaçao ao mercado. É o processo de reajustar o portifolio de forma a manter sempre uma alocação alvo. Por exemplo, digamos que uma estratégia de alocação indica que deve-se alocar 60% do portifolio no ativo A e 40% no ativo B e manter a posição constante; mesmo que depois de alguns dias o ativo A tenha apresentado melhor performance, digamos que agora representa 65% do seu portifolio contra 35% no ativo B, o rebalanceamento te leva a tirar investimento de A e mover para B até retornar ao alvo de 60%/40%. Rebalanceamento é uma estratégia de "compre na baixa / venda na alta", mesmo que pareça contra-intuitivo é uma estratégia altamente eficaz especialmente quando o mercado não apresenta fortes indícios de tendência.

### Indicadores de Risco

Avaliar um tipo de investimento apenas através do seu retorno passado não é a melhor alternativa. Felizmente, existem diferentes indicadores na literatura que representam a chance do retorno de um investimento ser diferente do inicialmente esperado, o que inclui a possibilidade de perder parte ou todo o valor aplicado. Indicadores de risco são resultados de cálculos feitos em dados históricos com o objetivo de quantificar o risco assumido para determinada posição de investimento. Por exemplo, um dos indicadores mais comuns usados por investidores é o Value-at-Risk (VaR), que é uma medida que indica a perda máxima, com certo grau de confiança, proveniente de oscilações de preços de mercado medidos dentro de um período determinado. Em outras palavras, o VaR(5%) representa a perda máxima potencial para o próximo período, com um grau de confiança de 95%. Existem diversos outros indicadores de risco que são especialmente relevantes e devem ser recalculados frequentemente a fim de manter um investimento com risco aceitável.


Para completar este artigo, mostraremos o cálculo do VaR para uma sequência de retornos. Utilizaremos dados históricos do preço da Bitcoin como exemplo. Os dados sao cortesia de https://coinmarketcap.com.

In [ ]:
close_df = pd.pivot_table(data, index='date', columns='symbol', values='close',
                              fill_value=0.0).loc['2015-01-01':]

Para começar, adquirimos os dados de retornos da Bitcoin de janeiro de 2015 até fevereiro de 2018, sendo estes as diferenças percentuais entre preços adjascentes. Assim:

In [ ]:
price = close_df.BTC.astype('f').dropna()
ret = price.pct_change().dropna()

plt.figure(figsize=(12,4))
plt.title("Preço da Bitcoin em USD")
plt.ylabel("preço")
plt.plot(pd.to_datetime(price.index), price)

plt.figure(figsize=(12,4))
plt.title("Retornos da Bitcoin em %")
plt.ylabel("retornos")
plt.plot(pd.to_datetime(ret.index), ret);

Desta sequência de retornos, podemos estimar uma distribuição empírica de probabilidades. Fazemos isto contando o número de ocorrências para cada magnitude. A frequência das ocorrencias é mostrada abaixo em um histograma.

In [ ]:
grey = .66, .66, .77
plt.figure(figsize=(13,6))
plt.title("Distribuição de retornos")
plt.ylabel("frequência")
plt.xlabel("magnitude")
plt.hist(ret, bins=50, normed=True, color=grey, edgecolor='none');

Com isto, já é possível estimar o VaR empírico da distribuição. Este é definido como o valor associado a um certo nível de confiança ($ alpha $), dada distribuição.
Para fazer isso, organizamos as amostras da menor para a maior e escolhemos a amostra que bate, em posição, com nosso intervalo de confiança, definido como $ 1 - alpha $.

In [ ]:
alpha1 = 0.05
alpha2 = 0.01

sorted_ret = np.sort(ret)

mu = ret.mean()
print("Média da distribuição: %.4f %%" % 
        mu
 )

var95 = abs(sorted_ret[int((1 - alpha1) * sorted_ret.shape[0])])
print("%.d%% Empirical VaR: %.2f %%" % ((
    1 - alpha1) * 100,
    var95 * 100)
 )

var99 = abs(sorted_ret[int((1 - alpha2) * sorted_ret.shape[0])])
print("%.d%% Empirical VaR: %.2f %%" % ((
    1 - alpha2) * 100,
    var99 * 100)
 )

grey = .66, .66, .77
plt.figure(figsize=(13,6))
plt.title("Distribuição de retornos")
plt.ylabel("frequência")
plt.xlabel("magnitude")
plt.hist(ret, bins=50, normed=True, color=grey, edgecolor='none');
plt.text(mu+0.003, 23, "Média: %.4f" % mu, color='black')
plt.plot([mu, mu], [0, 24], c='black')
plt.plot([-var95, -var95], [0, 6], c='b')
plt.text(-var95-0.01, 7.5, "95%% VaR", color='b')
plt.text(-var95-0.01, 6.5, var95, color='b')
plt.plot([-var99, -var99], [0, 4], c='r')
plt.text(-var99-0.01, 5.5, "99%% VaR", color='r')
plt.text(-var99-0.01, 4.5, var99, color='r');
plt.grid(linestyle='--')

Fazendo desta forma, não utilizamos de nenhum modelo hipotético, nem adicionamos nenhum conhecimento apriorístico a análise. Isso pode, ou não, ser vantajoso, dependendo de quão bem o modelo escolhido se ajusta aos dados. Na maioria dos casos, o cálculo do VaR é realizado assumindo-se uma distribuição normal de retornos. A distribuição normal assume caldas que decaem exponencialmente, e portanto não concentra muita probabilidade nas regiões mais distantes da média.

In [ ]:
dx = 0.0001  # resolution
x = np.arange(-5, 5, dx)
pdf = norm.pdf(x, 0, 1)

plt.figure(figsize=(13, 5))
plt.plot(x, pdf, 'b')
plt.title("Distribuição normal")
plt.ylabel("frequência")
plt.xlabel("magnitude")
plt.text(0.05, 0.42, "Média", color='black')
plt.plot([0, 0], [0, 0.43], c='black')
cred = norm.ppf(1-0.01) * 1 - 0
plt.plot([-cred, -cred], [0, 0.11], c='r')
plt.text(-cred-0.9, 0.1, "95%% cred", color='r')
plt.grid(linestyle='--')

Vamos prametrizar uma distribuição normal utilizando os dados de retornos da Bitcoin e analisar o resultado obtido para o VaR utilizando este modelo.

In [ ]:
mu_norm, sig_norm = norm.fit(ret)
dx = 0.0001  # resolution
x = np.arange(-0.2, 0.2, dx)
pdf = norm.pdf(x, mu_norm, sig_norm)

var95 = norm.ppf(1 - alpha1) * sig_norm - mu_norm
var99 = norm.ppf(1 - alpha2) * sig_norm - mu_norm

print("%.d%% Normal VaR: %.2f %%" % ((
    1 - alpha1) * 100,
    var95 * 100)
     )
print("%.d%% Normal VaR: %.2f %%" % ((
    1 - alpha2) * 100,
    var99 * 100)
     )

plt.figure(figsize=(13, 5))
plt.plot(x, pdf, 'b')
plt.title("Distribuição normal")
plt.ylabel("frequência")
plt.xlabel("magnitude")
plt.text(mu_norm+0.003, 10.5, "Média: %.4f" % mu_norm, color='black')
plt.plot([mu_norm, mu_norm], [0, 11], c='black')
plt.plot([-var95, -var95], [0, 4.2], c='r')
plt.text(-var95-0.035, 4.5, "95%% VaR", color='r');
plt.text(-var95-0.025, 4, "%.4f" % var95, color='r');
plt.plot([-var99, -var99], [0, 2.2], c='r')
plt.text(-var99-0.035, 2.5, "99%% VaR", color='r');
plt.text(-var99-0.025, 2, "%.4f" % var99, color='r');
plt.grid(linestyle='--')

Observa-se que os valores obtidos para o VaR utilizando a distribuição normal são menores do que aqueles obtidos utilizando-se a distribuição empírica. Isto ocorre porque a distribuição normal não é um bom modelo para retornos financeiros, principalmente para os do mercado de criptomoedas, dado que este exibe comportamentos extremos muito frequentemente. Podemos observar a falta de ajuste do modelo normal aos dados utilizando testes estatísticos. 